# 4.10: Coding Etiquette & Excel Reporting

## Contents

### 01. Importing libaries
### 02. Importing data
### 03. Check for data consistency
### 04. Address any PII issue the data set might have (Step 2)
### 05. Create a 'region' column based on the 'state' column (Step 3)
### 06. Exclusion flag for low revenue customers (Step 4 a)


## 01. Importing libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

## 02. Importing data

In [2]:
path = r'C:\Users\Sulo\Desktop\Careerfoundry\11.09.2023 Instacart Basket Analysis'

In [3]:
# Import the pickle file
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_customer_merge.pkl'))

### 03. Check for data consistency 

In [4]:
ords_prods_merge.isnull().sum()

order_id                        0
user_id                         0
order_amount                    0
orders_day_of_week              0
order_hour_of_day               0
days_since_prior_order          0
product_id                      0
add_to_cart_order               0
reordered                       0
product_name                    0
aisle_id                        0
department_id                   0
prices                       5127
price_range_loc                 0
busiest_days                    0
busiest_period_of_day_loc       0
max_order                       0
loyalty_flag                    0
avg_order                       0
spending_flag                   0
med_days                        0
order_frequency_flag            0
first_name                      0
surname                         0
gender                          0
state                           0
age                             0
date_joined                     0
n_dependants                    0
fam_status    

### The missing values of price are not computable and I didn't want to delete the rows of it, because it doesn't impact the analysis. 5127 missing values are 0.016% of the whole data and won't impact the results.

### 04. Address any PII issue the data set might have (Step 2)

In [5]:
# Check for columns that reveal sensitive information
ords_prods_merge.columns

Index(['order_id', 'user_id', 'order_amount', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', 'price_range_loc', 'busiest_days',
       'busiest_period_of_day_loc', 'max_order', 'loyalty_flag', 'avg_order',
       'spending_flag', 'med_days', 'order_frequency_flag', 'first_name',
       'surname', 'gender', 'state', 'age', 'date_joined', 'n_dependants',
       'fam_status', 'income'],
      dtype='object')

In [6]:
# Remove columns 'first name' and 'surname'
df_pii_safe = ords_prods_merge.drop(columns = ['first_name', 'surname'])

In [7]:
# From the previous exercise I knew that we had 31 columns and now it's 29 so dropping the columns worked
df_pii_safe.shape

(32404859, 29)

### 05. Create a 'region' column based on the 'state' column (Step 3)

In [8]:
# Assign states to regions 
Northeast =['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']
Midwest = ['Wisconsin', 'Michigan','Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']
South = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']
West = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

### We use the loc function to target every row/column and then we use the isin function alongside the variable
### to assign to a new column called 'region' based on the values of state. Needed to use the cheat sheet for that

In [9]:
df_pii_safe.loc[df_pii_safe['state'].isin(Northeast), 'region'] = 'Northeast'

In [10]:
df_pii_safe.loc[df_pii_safe['state'].isin(Midwest), 'region'] = 'Midwest'

In [11]:
df_pii_safe.loc[df_pii_safe['state'].isin(South), 'region'] = 'South'

In [12]:
df_pii_safe.loc[df_pii_safe['state'].isin(West), 'region'] = 'West'

In [13]:
# Check if the 'region' column got added
df_pii_safe.shape

(32404859, 30)

In [14]:
# Check again if it worked correctly
df_pii_safe.head()

,order_id,user_id,order_amount,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,med_days,order_frequency_flag,gender,state,age,date_joined,n_dependants,fam_status,income,region
0,2539329,1,1,2,8,7.0,196,1,0,Soda,...,20.0,Regular customer.,Female,Alabama,31,2/17/2019,3,married,40423,South
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,20.0,Regular customer.,Female,Alabama,31,2/17/2019,3,married,40423,South
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,20.0,Regular customer.,Female,Alabama,31,2/17/2019,3,married,40423,South
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,20.0,Regular customer.,Female,Alabama,31,2/17/2019,3,married,40423,South
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,20.0,Regular customer.,Female,Alabama,31,2/17/2019,3,married,40423,South


In [15]:
# Create a pivot table in phyton to compare the values
crosstab = pd.crosstab(df_pii_safe['region'], df_pii_safe['spending_flag'], dropna = False)

In [16]:
crosstab

spending_flag,High spender,Low spender
region,,
Midwest,155975,7441350
Northeast,108225,5614511
South,209691,10582194
West,160354,8132559


### There doesn't seem to be a big difference because based on the total customers the values are very similar

### 06. Exclusion flag for low revenue customers (Step 4 a)

In [17]:
# Create a column by deriving information from the 'max order' column to the column 'activity flag' for High-activity customers
df_pii_safe.loc[df_pii_safe['max_order'] >= 5, 'activity_flag'] = 'High-activity customer'

In [18]:
# Create a column by deriving information from the 'max order' column to the column 'activity flag' for Low-activity customers
df_pii_safe.loc[df_pii_safe['max_order'] < 5, 'activity_flag'] = 'Low-activity customer'

In [19]:
# Check if the values add up
df_pii_safe['activity_flag'].value_counts(dropna = False)

activity_flag
High-activity customer    30964564
Low-activity customer      1440295
Name: count, dtype: int64

In [20]:
# Set the data frame for low-activity customer
low_activity_customer = df_pii_safe[df_pii_safe['activity_flag'] == 'Low-activity customer']

In [21]:
# Check if the process worked
low_activity_customer.head()

,order_id,user_id,order_amount,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,order_frequency_flag,gender,state,age,date_joined,n_dependants,fam_status,income,region,activity_flag
1510,520620,120,1,3,11,7.0,196,2,0,Soda,...,Regular customer.,Female,Kentucky,54,3/2/2017,2,married,99219,South,Low-activity customer
1511,3273029,120,3,2,8,19.0,196,2,1,Soda,...,Regular customer.,Female,Kentucky,54,3/2/2017,2,married,99219,South,Low-activity customer
1512,520620,120,1,3,11,7.0,46149,1,0,Zero Calorie Cola,...,Regular customer.,Female,Kentucky,54,3/2/2017,2,married,99219,South,Low-activity customer
1513,3273029,120,3,2,8,19.0,46149,1,1,Zero Calorie Cola,...,Regular customer.,Female,Kentucky,54,3/2/2017,2,married,99219,South,Low-activity customer
1514,520620,120,1,3,11,7.0,26348,3,0,Mixed Fruit Fruit Snacks,...,Regular customer.,Female,Kentucky,54,3/2/2017,2,married,99219,South,Low-activity customer


In [22]:
# Export the unnecessary values
low_activity_customer.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'low_activity_customers.pkl'))

In [ ]:
# Export the current data 
df_pii_safe.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_update_v1.pkl'))